In [1]:
class PriorityQueue:
    """A priority queue that stores items with associated priorities.
    
    Items with higher priorities (lower numeric values) are dequeued before items with lower priorities.
    """

    def __init__(self):
        """Initializes an empty PriorityQueue instance with an internal list for storing items and priorities."""
        self.queue = []

    def enqueue(self, item, priority):
        """Adds an item to the queue with a specified priority.

        Args:
            item: The item to be added to the queue.
            priority: An integer or float representing the priority of the item.
                        Higher priority items (lower priority values) are placed before lower-priority ones.
        """
        new_element = (item, priority)
        inserted = False
        for index in range(len(self.queue)):
            if self.queue[index][1] < priority:  # Higher priority comes first
                self.queue.insert(index, new_element)
                inserted = True
                break
        if not inserted:
            self.queue.append(new_element)  # Add to the end if it has the lowest priority

    def dequeue(self):
        """Removes and returns the item with the highest priority from the queue.

        Returns:
            The item with the highest priority.

        Raises:
            Exception: If the priority queue is empty.
        """
        if not self.queue:
            raise Exception("Priority Queue is empty") 
        return self.queue.pop(0)[0]

    def is_empty(self):
        """Checks if the priority queue is empty.

        Returns:
            bool: True if the queue is empty, False otherwise.
        """
        return len(self.queue) == 0

def astar(start, goal, graph, heuristic):
    """A* search algorithm to find the shortest path between start and goal nodes in a weighted graph.

    Args:
        start: The starting node of the search.
        goal: The goal node of the search.
        graph: A dictionary representing the graph, where each key is a node, and values are lists of 
                (neighbor, cost) pairs.
        heuristic: A function that estimates the distance from any node to the goal.

    Returns:
        list: A list of nodes representing the shortest path from start to goal.

    Raises:
        Exception: If no path is found from start to goal.
    """
    open_set = PriorityQueue()
    open_set.enqueue(start, 0)
    
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while not open_set.is_empty():
        current = open_set.dequeue()
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  # Return reversed path
        
        for neighbor, cost in graph.get(current, []):
            tentative_g_score = g_score[current] + cost
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                open_set.enqueue(neighbor, -f_score[neighbor])  # Use -f_score to prioritize lower f_score
    
    raise Exception("No path found from start to goal")

def heuristic(node, goal):
    """Calculates the squared Euclidean distance between the current node and the goal node.

    Args:
        node: A tuple representing the (x, y) coordinates of the current node.
        goal: A tuple representing the (x, y) coordinates of the goal node.

    Returns:
        float: The squared Euclidean distance between the node and the goal.
    """
    x1, y1 = node
    x2, y2 = goal
    return (x1 - x2) ** 2 + (y1 - y2) ** 2


In [2]:
def create_grid_graph(width, height):
    """Creates a grid graph where each node represents a coordinate (x, y) on a grid,
    and each node has neighbors in the four cardinal directions with a movement cost of 1.

    Args:
        width (int): The width of the grid.
        height (int): The height of the grid.

    Returns:
        dict: A dictionary representing the graph, where each key is a (x, y) node coordinate,
                and the value is a list of tuples containing neighboring coordinates and movement costs.
    """
    graph = {}
    for x in range(width):
        for y in range(height):
            neighbors = []
            if x > 0:
                neighbors.append(((x - 1, y), 1))
            if x < width - 1:
                neighbors.append(((x + 1, y), 1))
            if y > 0:
                neighbors.append(((x, y - 1), 1))
            if y < height - 1:
                neighbors.append(((x, y + 1), 1))
            graph[(x, y)] = neighbors
    return graph

def heuristic(node, goal):
    """Calculates the Manhattan distance between two nodes on a grid, useful as a heuristic in pathfinding.

    Args:
        node (tuple): The current node's (x, y) coordinates.
        goal (tuple): The goal node's (x, y) coordinates.

    Returns:
        int: The Manhattan distance between the current node and the goal node.
    """
    x1, y1 = node
    x2, y2 = goal
    return abs(x1 - x2) + abs(y1 - y2)

def test_astar():
    """Tests the A* algorithm on a 5x5 grid from a start position at (0, 0) to a goal at (4, 4).
    
    The expected shortest path length from (0, 0) to (4, 4) is 8 (Manhattan distance).
    
    Raises:
        AssertionError: If the A* path does not start at the start position, end at the goal, or match
                        the expected path length.
    """
    width, height = 5, 5
    start, goal = (0, 0), (4, 4)
    graph = create_grid_graph(width, height)
    
    try:
        path = astar(start, goal, graph, heuristic)
        print("Path found by A*:", path)
        print("Path length:", len(path) - 1)
        assert path[0] == start, "Path should start at the start position."
        assert path[-1] == goal, "Path should end at the goal position."
        assert len(path) - 1 == heuristic(start, goal), "Path length should match the heuristic distance."
        print("Test passed!")
    except Exception as e:
        print("Test failed:", e)

test_astar()


Path found by A*: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
Path length: 8
Test passed!
